## 1. Info

In [1]:
import os 

DATAPATH = './FW&BW_Rawdata/'

In [2]:
files = os.listdir(DATAPATH + 'PD')
fnames = set()

for f in files:
    fnames.add(f[:3])
    
print('PD:', len(fnames))

PD: 81


In [3]:
files = os.listdir(DATAPATH + 'Controls')
fnames = set()

for f in files:
    fnames.add(f[:3])
    
print('Controls:', len(fnames))

Controls: 15


## 2. Prep

In [4]:
import pandas as pd

RAWDATAPATH = './FW&BW_Rawdata/PD'
DATASETPATH = './dataset/'

if os.path.exists(DATASETPATH) == False:
    os.mkdir(DATASETPATH)

# TARGET_FILE = 'AMJ_BW1.csv' 

In [5]:
def prepData(TARGET_PATH, ptype):
    # 1. 2줄 제외하고 읽기
    dff = pd.read_csv(TARGET_PATH, skiprows=2, encoding='utf-8')
    
    # 2. 전체 none인 컬럼 제외
    while dff[list(dff.columns)[-1]].isna().all():
        dff.drop(list(dff.columns)[-1], axis=1, inplace=True)
        
    # 3. 단위가 mm인 데이터만 추출
    target = [0, 1]
    target += list(filter(lambda x : list(dff.iloc[1] == 'mm')[x], range(len(list(dff.iloc[1])))))
    dff = dff.iloc[:, target]
    dff.drop(dff.index[1], inplace=True)
    
    # 4. 컬럼 이름 변경하기 

    pList = dff.columns
    colList = dff.iloc[0]

    NAME = None

    pNames = []
    for p in pList:
        if not p.startswith('Unnamed'):
            if NAME == None: NAME = p.split(':')[0]
            pNames.append(p.split(':')[1])
            
#     print(pNames)
#     print(colList[2:])

#     assert len(pNames) == 39 and len(colList[2:]) == 39*3

    columns = []
    columns.append(colList[0])
    columns.append(colList[1])
    for idx, col in enumerate(colList[2:]):
        i = int(idx / 3)
        eachColName = pNames[i] + '_' + col
        columns.append(eachColName)

    assert len(columns) == len(list(dff.columns))

    dff.drop(dff.index[0], inplace=True)
    dff.reset_index(drop=True, inplace=True)

    dff.columns = columns
    
    if os.path.exists(DATASETPATH + ptype) == False:
        os.mkdir(DATASETPATH + ptype)
        
    dff.to_csv(os.path.join(DATASETPATH + ptype, 'PREP_'+TARGET_FILE), encoding='utf-8', index=False)
    
#     print(dff.head())

In [6]:
RAWDATAPATH = './FW&BW_Rawdata/PD'
for TARGET_FILE in os.listdir(RAWDATAPATH):
    if TARGET_FILE.endswith('.csv'):
        prepData(os.path.join(RAWDATAPATH, TARGET_FILE), RAWDATAPATH.split('/')[-1])

KeyboardInterrupt: 

In [ ]:
RAWDATAPATH = './FW&BW_Rawdata/Controls'
for TARGET_FILE in os.listdir(RAWDATAPATH):
    if TARGET_FILE.endswith('.csv'):
        prepData(os.path.join(RAWDATAPATH, TARGET_FILE), RAWDATAPATH.split('/')[-1])

In [108]:
RAWDATAPATH = './FW&BW_Rawdata/Controls(추가)'
for TARGET_FILE in os.listdir(RAWDATAPATH):
    if TARGET_FILE.endswith('.csv'):
        prepData(os.path.join(RAWDATAPATH, TARGET_FILE), RAWDATAPATH.split('/')[-1])

In [109]:
# 데이터 확인
files = os.listdir(DATASETPATH + 'PD')
fnames = set()

for f in files:
    if f.endswith('.csv'):
        fnames.add(f.split('_')[1])
    
print('PD:', len(fnames))

files = os.listdir(DATASETPATH + 'Controls')
fnames = set()

for f in files:
    if f.endswith('.csv'):
        fnames.add(f.split('_')[1])
    
print('Controls:', len(fnames))

PD: 83
Controls: 22


## 3. 환자별 데이터 테이블 생성

In [7]:
import os

DATASETPATH = './dataset/'
DATA_CONTROL = os.path.join(DATASETPATH , 'Controls')
DATA_PD = os.path.join(DATASETPATH, 'PD') 

In [43]:
patient = []
category = []
cntFW = []
cntBW = []
pathFW = []
pathBW = []

In [44]:
# Controls
files = os.listdir(DATA_CONTROL)
files.sort()

prevName = files[0].split('_')[1]
idx = 0

while idx < len(files):
    if files[idx].endswith('.csv'):
        files[idx].replace('.csv','')
        _, pName, bwfw = files[idx].split('_')
        bw = 0
        fw = 0
        pFW = []
        pBW = []
        while pName == prevName:
            if bwfw.startswith('BW'):
                bw += 1
                pBW.append(os.path.join(DATA_CONTROL, files[idx]))
            else:
                fw += 1
                pFW.append(os.path.join(DATA_CONTROL, files[idx]))
#             print(f.split('_')[1], f.split('_')[2].split('.csv')[0])
            idx += 1
            if idx == len(files): break
            _, pName, bwfw = files[idx].split('_')
        
        patient.append(prevName)
        category.append('Controls')
        cntFW.append(fw)
        cntBW.append(bw)
        pathFW.append(pFW)
        pathBW.append(pBW)
        
        prevName = pName

In [45]:
# PD
files = os.listdir(DATA_PD)
files.sort()

prevName = files[0].split('_')[1]
idx = 0

while idx < len(files):
    if files[idx].endswith('.csv'):
        files[idx].replace('.csv','')
        if len(files[idx].split('_')) == 3:
            _, pName, bwfw = files[idx].split('_')
        else:
            _, pName, bwfw, _ = files[idx].split('_')
        bw = 0
        fw = 0
        pFW = []
        pBW = []
        while pName == prevName:
            if bwfw.startswith('BW'):
                bw += 1
                pBW.append(os.path.join(DATA_CONTROL, files[idx]))
            else:
                fw += 1
                pFW.append(os.path.join(DATA_CONTROL, files[idx]))
#             print(f.split('_')[1], f.split('_')[2].split('.csv')[0])
            idx += 1
            if idx == len(files): break
#             print(files[idx])
            if len(files[idx].split('_')) == 3:
                _, pName, bwfw = files[idx].split('_')
            else:
                _, pName, bwfw, _ = files[idx].split('_')
                   
        patient.append(prevName)
        category.append('PD')
        cntFW.append(fw)
        cntBW.append(bw)
        
        prevName = pName

In [49]:
df = pd.DataFrame()
df['Patient'] = patient
df['Category'] = category
df['cntFW'] = cntFW
df['cntBW'] = cntBW
# df['pathFW'] = pathFW
# df['pathBW'] = pathBW

In [50]:
df

,Patient,Category,cntFW,cntBW
0,BHY,Controls,3,3
1,CHH,Controls,3,3
2,HSH,Controls,3,3
3,JHY,Controls,3,3
4,JJG,Controls,3,3
...,...,...,...,...
100,YJM,PD,3,3
101,YKJ,PD,3,3
102,YKS,PD,3,3
103,YSC,PD,3,3


In [39]:
df.to_csv(os.path.join(DATASETPATH, 'patients.csv'), encoding='utf-8', index=False)